In [ ]:
import os 

# Run global_variables, synthesize_functions and mlp_model
%run ~/violin-renderer/src/global_variables.ipynb
%run ~/violin-renderer/src/models/lstm/lstm_model.ipynb
%run ~/violin-renderer/src/synthesize/synthesize_functions.ipynb
%run ~/violin-renderer/src/models/lstm/data_processing.ipynb

In [ ]:
testing_X, testing_y = load_testing_data()

In [ ]:
def trim_sequence(input_song, true_song):
    all_source_inputs = []
    all_ground_truths = []

    for note_source, note_truth in zip(input_song, true_song):
        if note_source[1] > 64 * 24:
            break
            
        note_source = list(note_source)
        note_truth = list(note_truth)
        
        note_source[0] = scale_timings(note_source[0])
        note_source[1] = scale_timings(note_source[1])
        note_source[2] = scale_pitch(note_source[2])
        
        all_source_inputs.append(note_source)
        all_ground_truths.append(note_truth)

    return all_source_inputs, all_ground_truths

In [ ]:
# Create an output directory in /src, generate all results from the test dataset and synthesize all of them
# Setting up paths and output folders
SOURCE_INPUT_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/source-input'
TEMPO_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/tempos'
OUTPUT_PATH = HOME_PATH + '/src/lstm-output'
composer_directories = os.listdir(SOURCE_INPUT_PATH)

In [ ]:
input_dim = 3
hidden_dim = 32
layer_dim = 2
output_dim = 2

# initialize the MLP
model = LSTM(input_dim, hidden_dim, layer_dim, output_dim)

model.load_state_dict(torch.load(HOME_PATH + '/src/models/lstm/lstm.pt'))

In [ ]:
# Get generated output values from all songs
results = {}

for input_path, true_path in zip(testing_X, testing_y):
    trimmed_notes = []
    trimmed_notes, trimmed_truth = trim_sequence(testing_X[input_path], testing_y[true_path])
    trimmed_notes = [trimmed_notes]
    trimmed_notes = torch.Tensor(trimmed_notes)
    length = [len(trimmed_notes[0])]
    length = torch.Tensor(length)

    pred = model(trimmed_notes, length)
    pred = pred.tolist()
    results[input_path] = pred

print(results)

In [ ]:
# Synthesize
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

for notes_path, generated_results in results.items():
    name = '/' + notes_path.split('/')[-1]
    name = name.replace('.csv', '')
    generated_timings = [(note[0], note[1]) for note in generated_results[0]]
    synthesize_generated_output(notes_path, generated_timings, OUTPUT_PATH, name)